In [1]:
#import librairies 

import tensorflow as tf
import pandas as pd 
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import progressbar
import time
from time import process_time
import sys
import os
import random
import gc
    
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import f1_score,classification_report,roc_auc_score,precision_score,recall_score, precision_recall_fscore_support 
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn_som.som import SOM
import networkx as nx
from networkx.algorithms import approximation

pd.set_option('display.max_columns', 100)

from GEM.gem.utils      import graph_util, plot_util
from GEM.gem.evaluation import visualize_embedding as viz
from GEM.gem.evaluation import evaluate_graph_reconstruction as gr
from GEM.gem.embedding.gf       import GraphFactorization
#from GEM.gem.embedding.sdne     import SDNE
#from argparse import ArgumentParser
#from GraphEmbedding.ge import DeepWalk
#from GraphEmbedding.ge import SDNE
from karateclub.graph_embedding import Graph2Vec
from karateclub.node_embedding.neighbourhood import HOPE
from karateclub.node_embedding.neighbourhood import DeepWalk




num_frame=200#arbitraire , a tester plus serieusement

print("import terminé a :",time.strftime("%H:%M:%S", time.localtime()))

import terminé a : 09:18:07


In [9]:
#ce dictionaire contien le nom de chaques colones avec la valeur du f1_score
#qu'ils on obtenu apres etre ajouté au data_set
#initialisé a 0
#-1,name1,name2,w,b

def convert_dist_to_zero_and_ones(data_set_dist, threshold=0.3):
    new_data_set = data_set_dist < threshold
    return new_data_set.astype(int)

def create_pair_dict(columns_list=[]):
    if (columns_list==[]):
        print("you forgot to input a list of columns")
        return []
    pair_dict=dict()
    for name1 in columns_list:
        for name2 in columns_list:# oui ce n'est pas optimisé
            pair_names=name1+"_"+name2
            reverse_pair_names=name2+"_"+name1
            if name1!=name2 and pair_names not in pair_dict.keys() and reverse_pair_names not in pair_dict.keys():
                pair_dict[pair_names]=[-1,name1,name2,0,0]
    
    return pair_dict

def create_monstruous_pair_data_set_deprecated(pair_dict,panda_data_set):
    columns_list = panda_data_set.columns.tolist()
    if (columns_list==[]):
        print("you forgot to input a data_set")
        return {},[]
    if (pair_dict=={}):
        print("you forgot to input a dict")
        return {},[]
    #pair_dict=dict()
    process_bar = progressbar.ProgressBar().start(max_value=len(pair_dict));i=0
    monstruous_pair_data_set=[]
    for pair in pair_dict.items():
        pair_dict[pair[0]][0]=i
        #print(pair[1][0],pair[1][1],pair[1][2])# value name1,name2
        tmp_data_set=panda_data_set[[pair[1][1],pair[1][2]]]
        tmp_w,tmp_b=linear_regresion(tmp_data_set)
        pair_dict[pair[0]][3]=tmp_w
        pair_dict[pair[0]][4]=tmp_b
        monstruous_pair_data_set.append(distances(tmp_data_set,tmp_w,tmp_b))
        process_bar.update(i);i+=1
    #the monstruous data_set should contain 0 and 1
    return pair_dict,monstruous_pair_data_set

def create_monstruous_pair_data_set(pair_dict,panda_data_set):
    columns_list = panda_data_set.columns.tolist()
    if (columns_list==[]):
        print("you forgot to input a data_set")
        return {},[]
    if (pair_dict=={}):
        print("you forgot to input a dict")
        return {},[]
    #pair_dict=dict()
    process_bar = progressbar.ProgressBar().start(max_value=len(pair_dict));i=0
    monstruous_pair_data_set=np.array([])
    for pair in pair_dict.items():
        #print(pair[1][0],pair[1][1],pair[1][2])# id,name1,name2,w,b
        pair_dict[pair[0]][0]=i
        tmp_data_set=panda_data_set[[pair[1][1],pair[1][2]]]
        tmp_w,tmp_b=linear_regresion(tmp_data_set)
        pair_dict[pair[0]][3]=tmp_w
        pair_dict[pair[0]][4]=tmp_b
        
        if(monstruous_pair_data_set.size ==0):
            monstruous_pair_data_set=distances_expand(tmp_data_set,tmp_w,tmp_b)
        else:
            monstruous_pair_data_set=np.concatenate((monstruous_pair_data_set,distances_expand(tmp_data_set,tmp_w,tmp_b)),axis=1)
        process_bar.update(i);i+=1
    #the monstruous data_set should contain 0 and 1
    return pair_dict,monstruous_pair_data_set

def make_one_graph(line ,pair_dict):
    g=nx.Graph()
    
    for pair in pair_dict.items():
        #print(pair[1][0],pair[1][1],pair[1][2])# id,name1,name2,w,b
        i= pair[1][0]
        arc_val=line[i]

        if arc_val ==1:
            name_1=pair[1][1]
            name_2=pair[1][2]
            g.add_edge(name_1,name_2)
    
    return g

def create_graphs(monstruous_binary_np_data_set,pair_dict):
    start = time.time()
    graph_list=[]
    process_bar = progressbar.ProgressBar().start(max_value=len(monstruous_binary_np_data_set));i=0
   
    for line in monstruous_binary_np_data_set:
        #print(type(line),line.shape)

        graph_list.append( make_one_graph(line ,pair_dict))
        process_bar.update(i);i+=1
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
 
    return graph_list

def save_np(np_array,name):
    if(".npy" not in name):
        print("you save a numpy array in a file.npy")
        return 0
    np.save(name, np_array)

def load_np(file_name):
    return np.load(file_name)

    
############################################## liste des options d'extractions
def graph_max_degree(g):
    degrees = [val for (node, val) in g.degree()]
    maxd=max(degrees)
    return maxd
def geodesic_dist(graph):
    return nx.average_shortest_path_length(graph)

#special thanks to Francisco A. Rodrigues, University of São Paulo.
# http://conteudo.icmc.usp.br/pessoas/francisco
def degree_distribution(G):
    vk = dict(G.degree())
    vk = list(vk.values()) # we get only the degree values
    maxk = np.max(vk)
    mink = np.min(min)
    kvalues= np.arange(0,maxk+1) # possible values of k
    Pk = np.zeros(maxk+1) # P(k)
    for k in vk:
        Pk[k] = Pk[k] + 1
    Pk = Pk/sum(Pk) # the sum of the elements of P(k) must to be equal to one
    return kvalues,Pk
def shannon_entropy(G):
    k,Pk = degree_distribution(G)
    H = 0
    for p in Pk:
        if(p > 0):
            H = H - p*math.log(p, 2)
    return H

def contain_meso_scale(graph):
    
    return False    




def extract_info_from_one_graph(g, to_do_list):
    graph_property=np.array([])
    calc_result=0
    
    if("max_degree" in to_do_list):
        #print (" max degree")
        calc_result=graph_max_degree(g)
        graph_property=np.append(graph_property,calc_result)
    if("assortativity" in to_do_list):
        calc_result = nx.degree_assortativity_coefficient(g)
        graph_property=np.append(graph_property,calc_result)
    if("clustering" in to_do_list):
        calc_result= approximation.average_clustering(g, trials=1000, seed=10)
        graph_property=np.append(graph_property,calc_result)
    if("global_efficiency" in to_do_list):
        calc_result= nx.global_efficiency(g)
        graph_property=np.append(graph_property,calc_result)   
    if("geodesic_dist" in to_do_list):
        if nx.is_connected(g):
            calc_result=geodesic_dist(g)
        else:
            calc_result=len(g.nodes()) # = la dist la plus grande
        graph_property=np.append(graph_property,calc_result)
    if("contain_meso_scale" in to_do_list):#to do
        calc_result=contain_meso_scale(g)
        graph_property=np.append(graph_property,calc_result)
    if("Shannon_entropy" in to_do_list):#to do
        calc_result=shannon_entropy(g)
        graph_property=np.append(graph_property,calc_result)  
    #change this ()
    #            ()
    #            ()
    #      to this ()()()
    return np.expand_dims(graph_property,axis=0)

def extract_info_from_graphs(graph_list, to_do_list):
    
    monstruous_info_data_set=np.array([])
    process_bar = progressbar.ProgressBar().start(max_value=len(graph_list));i=0
   
    
    for graph in graph_list:
        
        if(monstruous_info_data_set.size ==0):
            monstruous_info_data_set=extract_info_from_one_graph(graph, to_do_list)
        else:
            monstruous_info_data_set=np.concatenate((monstruous_info_data_set,extract_info_from_one_graph(graph, to_do_list)),axis=0)
        process_bar.update(i);i+=1
    
    return monstruous_info_data_set



def create_graphs_to_delete(monstruous_binary_np_data_set,pair_dict):
    start = time.time()
    graph_list=[]
    process_bar = progressbar.ProgressBar().start(max_value=len(monstruous_binary_np_data_set));i=0
   
    for line in monstruous_binary_np_data_set:
        #print(type(line),line.shape)

        graph_list.append( make_one_graph(line ,pair_dict))
        process_bar.update(i);i+=1
    print("---graph construction = %s seconds ---" % (time.time() - start));start = time.time()
    return False


def extract_info_from_binary_data_set(monstruous_binary_np_data_set,pair_dict, to_do_list):
    start_time = process_time() 
    process_bar = progressbar.ProgressBar().start(max_value=len(monstruous_binary_np_data_set));i=0
    monstruous_info_data_set=np.array([])
    counter = 0
    contruct_time=0
    calcul_time=0
    for line in monstruous_binary_np_data_set:
        in_loop_time = process_time()
        graph=make_one_graph(line ,pair_dict)
        contruct_time = contruct_time+ process_time()  - in_loop_time
        
        in_loop_time = process_time()
        if(monstruous_info_data_set.size ==0):
            monstruous_info_data_set=extract_info_from_one_graph(graph, to_do_list)
        else:
            monstruous_info_data_set=np.concatenate((monstruous_info_data_set,extract_info_from_one_graph(graph, to_do_list)),axis=0)
        calcul_time = calcul_time+ process_time()  - in_loop_time
        process_bar.update(i);i+=1
        
        #####################################################################
        del graph # je delete le graph pour etre sur de liberer la memoire
        #gc.collect() #je n'appelle pas le garbageColector = cout 22 heures
        #calcul_time =  521.984375 contruct_time 59.3125
        #counter=counter+1
        #if(counter == 100000):  
            #print(process_time()  - start_time, "seconds");start_time = process_time()
            #print("calcul_time = ",calcul_time,"contruct_time",contruct_time)
            #return monstruous_info_data_set
            
        
    
    print("---graph extraction = %s seconds ---" % (process_time()  - start_time));start_time = process_time()
    return monstruous_info_data_set

# Function to find distance line equation ax + by + c =0
#y=mx+c  ----> mx -y +c =0  ---> b=-1 ,a=weight ,c=bias
def shortest_distance(x1, y1, a, b, c):
    
    d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
    #print("Perpendicular distance is ",d)
    return d

def distances(data_set,w,b):
    np_data=data_set.to_numpy()
    np_dist=[]
    for line in np_data:
        x1=line[0]
        y1=line[1]
        np_dist.append(shortest_distance(x1, y1, w, -1, b))
    
    return np.array(np_dist)

def distances_expand(data_set,w,b):
    np_data=data_set.to_numpy()
    np_dist=[]
    for line in np_data:
        x1=line[0]
        y1=line[1]
        np_dist.append(shortest_distance(x1, y1, w, -1, b))
    
    return np.expand_dims( np.array(np_dist)  ,axis=1)
# y = mx +b
def line_equation_from_two_points(x1,y1,x2,y2):
    m=(y2-y1)/(x2-x1)
    b=y1-m*x1
    return m,b



def shortest_distance_test():
    print(shortest_distance(0, 0, 1, -1, 1))
    print(shortest_distance(-1, 0, 1, -1, 1))
    print(shortest_distance(1, 0, 1, -1, 1))
    print(math.sqrt(2))
    print(shortest_distance(3, 0, 1, -1, 1))
    print(math.sqrt(2*2+2*2))


#https://www.machinelearningplus.com/deep-learning/linear-regression-tensorflow/
###############################################################################
################# partie linear regression ####################################
###############################################################################
###############################################################################
###############################################################################

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def lin_reg_bootstrap(panda_data_set,nb_iter=1000):
    columns_list = panda_data_set.columns.tolist()
    x_train=NormalizeData(panda_data_set[columns_list[0]].to_numpy())
    y_train=NormalizeData(panda_data_set[columns_list[1]].to_numpy())
    data_set_size=len(x_train)
    m=0
    b=0
    counter=0
    for i in range(nb_iter):
        p1=random.randint(0, data_set_size-1)
        p2=random.randint(0, data_set_size-1)
        
        x1=x_train[p1]
        y1=y_train[p1]        
        x2=x_train[p2]
        y2=y_train[p2]
        if((x2-x1)!=0):
            m_tmp,b_tmp=line_equation_from_two_points(x1,y1,x2,y2)
            m=m+m_tmp
            b=b+b_tmp
        else:
            counter=counter+1
    m=m/(nb_iter-counter)
    b=b/(nb_iter-counter)
    return m,b
    
def linreg(x,weight,bias):#x is a np list
    y = weight*x + bias
    return y

# Define loss function (MSE)
def squared_error(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

def not_squared_error(y_pred, y_true):
    return tf.reduce_mean(y_pred - y_true)

def linear_regresion(panda_data_set,boot_iter=1000):
    columns_list = panda_data_set.columns.tolist()
    #x_train=panda_data_set[columns_list[0]].to_numpy()
    #y_train=panda_data_set[columns_list[1]].to_numpy()
    x_train=NormalizeData(panda_data_set[columns_list[0]].to_numpy())
    y_train=NormalizeData(panda_data_set[columns_list[1]].to_numpy())
    #print("x_train max =",np.amax(x_train, axis=0))
    #print("y_train max =",np.amax(y_train, axis=0))
    learning_rate = 0.01
    # Number of loops for training through all your data to update the parameters
    training_epochs = 100
    
    boot_weight,boot_bias=lin_reg_bootstrap(panda_data_set,nb_iter=boot_iter)
    weight = tf.Variable(boot_weight)
    bias   = tf.Variable(boot_bias)
    #weight = tf.Variable(0.)
    #bias   = tf.Variable(0.)
    
    
    for epoch in range(training_epochs):

        # Compute loss within Gradient Tape context
        with tf.GradientTape() as tape:
            y_predicted = linreg(x_train,weight,bias)
            #loss = squared_error(y_predicted, y_train)
            loss = not_squared_error(y_predicted, y_train)
            # Get gradients
            gradients = tape.gradient(loss, [weight,bias])

            # Adjust weights
            weight.assign_sub(gradients[0]*learning_rate)
            bias.assign_sub(gradients[1]*learning_rate)

    
    
    return weight.numpy(),bias.numpy()

print("fonctions declaré a :",time.strftime("%H:%M:%S", time.localtime()))



fonctions declaré a : 11:14:24


In [3]:
#import le data_set complet --- import data_set = 28.00195622444153 seconds ---
start = time.time()

data_file= os.path.abspath('../../data')
full_path=data_file+'\\'+'fraudTrain.csv'   # unmodified train set
train_df=pd.read_csv(full_path)
full_path=data_file+'\\'+'fraudTest.csv'
test_df=pd.read_csv(full_path)



full_path=data_file+'\\'+'X_train_1_2_svm.csv'
xtrain_transformed_complique=pd.read_csv(full_path)
ytrain_transformed_complique=train_df['is_fraud'].iloc[:int(len(train_df)*0.8)]

full_path=data_file+'\\'+'X_val_1_2_svm.csv'
xval_transformed_complique=pd.read_csv(full_path)
yval_transformed_complique=train_df['is_fraud'].iloc[len(ytrain_transformed_complique):]


full_path=data_file+'\\'+'X_test_1_2_svm.csv'
xtest_transformed_complique=pd.read_csv(full_path)
ytest_transformed_complique=test_df['is_fraud']


cols = xtrain_transformed_complique.columns.tolist()
print(cols)
print("--- import data_set = %s seconds ---" % (time.time() - start));start = time.time()


['Unnamed: 0', 'merchant', 'category', 'amt', 'gender', 'state', 'zip', 'lat', 'long', 'city_pop', 'dob', 'unix_time', 'merch_lat', 'merch_long', 'delta_time', 'delta_amt', 'delta_time_category', 'delta_amt_category', 'delta_time_merchant', 'delta_amt_merchant', 'avg_amt', 'delta_avg_amt', 'avg_amt_category', 'delta_avg_amt_category', 'avg_amt_merchant', 'avg_amt_state', 'avg_amt_city', 'avg_amt_job', 'delta_avg_amt_category_job', 'month', 'day', 'hour']
--- import data_set = 20.039904832839966 seconds ---


In [4]:
dict_378 = create_pair_dict(cols)
print (len(dict_378))
print(32*32)
print(len(cols))

496
1024
32


In [6]:
#create data_set  ---create_monstruous_pair_data_set = ?????.00195622444153 seconds ---
start = time.time()
train_dict,train_graph_dist = create_monstruous_pair_data_set(dict_378,xtrain_transformed_complique)
val_dict,val_graph_dist = create_monstruous_pair_data_set(dict_378,xval_transformed_complique)
test_dict,test_graph_dist = create_monstruous_pair_data_set(dict_378,xtest_transformed_complique)
print("--- create_graph_dist = %s seconds ---" % (time.time() - start));start = time.time()


 99% (495 of 496) |##################### | Elapsed Time: 0:24:41 ETA:   0:00:04

In [7]:
train_graph_binary=convert_dist_to_zero_and_ones(train_graph_dist)
val_graph_binary=convert_dist_to_zero_and_ones(val_graph_dist)
test_graph_binary=convert_dist_to_zero_and_ones(test_graph_dist)
print("--- convert_dist_to_zero_and_ones = %s seconds ---" % (time.time() - start));start = time.time()


In [10]:
#crash a 22% pour cause de memoire ?
#graph_list=create_graphs(M_le_binary,dict_378_binary)

# 1000 iter -->90 sec
#to_do_list=["max_degree","assortativity","Clustering"]

# 1000 iter --> 91  sec
to_do_list=["max_degree","assortativity","Clustering","global_efficiency","geodesic_dist","Shannon_entropy"]

#2h 7%
train_graph_properties=extract_info_from_binary_data_set(train_graph_binary,train_dict,to_do_list)
val_graph_properties=extract_info_from_binary_data_set(val_graph_binary,val_dict,to_do_list)
test_graph_properties=extract_info_from_binary_data_set(test_graph_binary,test_dict,to_do_list)

  0% (21 of 259335) |                    | Elapsed Time: 0:00:00 ETA:   0:21:02

---graph extraction = 14324.84375 seconds ---


  0% (20 of 555719) |                    | Elapsed Time: 0:00:00 ETA:  00:00:00

---graph extraction = 1953.75 seconds ---


 99% (555717 of 555719) |############### | Elapsed Time: 1:32:15 ETA:   0:00:00

---graph extraction = 5520.125 seconds ---


In [11]:
## ajouter des colones


print(xtrain_transformed_complique.shape)
print(train_graph_properties.shape)

X_train_plus_graph =np.concatenate((xtrain_transformed_complique,train_graph_properties),axis=1)
print(X_train_plus_graph.shape)

X_val_plus_graph =np.concatenate((xval_transformed_complique,val_graph_properties),axis=1)
X_test_plus_graph =np.concatenate((xtest_transformed_complique,test_graph_properties),axis=1)


(1037340, 32)
(1037340, 5)
(1037340, 37)


In [12]:
# test avec xgboost
from xgboost import XGBClassifier

In [13]:
xgb = XGBClassifier(n_estimators=50, gamma=0.05,eta=0.05,max_depth=7, n_jobs=16)

xgb.fit(xtrain_transformed_complique,ytrain_transformed_complique)
Y_train_pred=xgb.predict(xtrain_transformed_complique)
Y_test_pred=xgb.predict(xtest_transformed_complique)
print(classification_report(ytrain_transformed_complique,Y_train_pred))
print(classification_report(ytest_transformed_complique,Y_test_pred))

C:\Users\benjamin.marty\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:23:16] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.98      0.83      0.90      5968

    accuracy                           1.00   1037340
   macro avg       0.99      0.91      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.95      0.79      0.86      2145

    accuracy                           1.00    555719
   macro avg       0.98      0.89      0.93    555719
weighted avg       1.00      1.00      1.00    555719



In [14]:
X_train_plus_graph
xgb.fit(X_train_plus_graph,ytrain_transformed_complique)
Y_train_pred_graph=xgb.predict(X_train_plus_graph)
Y_test_pred_graph=xgb.predict(X_test_plus_graph)
print(classification_report(ytrain_transformed_complique,Y_train_pred_graph))
print(classification_report(ytest_transformed_complique,Y_test_pred_graph))

print("---XGboost avec modification = %s seconds ---" % (time.time() - start));start = time.time()


C:\Users\benjamin.marty\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:25:00] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       0.98      0.83      0.90      5968

    accuracy                           1.00   1037340
   macro avg       0.99      0.91      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.94      0.79      0.86      2145

    accuracy                           1.00    555719
   macro avg       0.97      0.89      0.93    555719
weighted avg       1.00      1.00      1.00    555719

---XGboost avec modification = 29314.38864517212 seconds ---


In [15]:
cl=classification_report(ytest_transformed_complique,Y_test_pred,output_dict=True)
cl2=classification_report(ytest_transformed_complique,Y_test_pred_graph,output_dict=True)
recall_diff=(cl2["1"]["recall"]-cl["1"]["recall"])
f1_diff=(cl2["1"]["f1-score"]-cl["1"]["f1-score"])
print("xgboost recall diff = %.4f" % recall_diff)
print("xgboost f1-score diff = %.4f" % f1_diff)


xgboost recall diff = -0.0014
xgboost f1-score diff = -0.0035


In [16]:
start = time.time()

from sklearn.ensemble import RandomForestClassifier
max_depth = 15
rf = RandomForestClassifier(random_state=0, max_depth=max_depth, min_samples_leaf= 3, min_samples_split= 2, n_estimators= 140, verbose=1, n_jobs=16)
rf.fit(xtrain_transformed_complique, ytrain_transformed_complique)
# Prediction for the training/validation set
y_train_pred = rf.predict(xtrain_transformed_complique)
y_test_pred = rf.predict(xtest_transformed_complique)

print("---RandomForest sans ajout = %s seconds ---" % (time.time() - start));start = time.time()

max_depth = 15
rf = RandomForestClassifier(random_state=0, max_depth=max_depth, min_samples_leaf= 3, min_samples_split= 2, n_estimators= 140, verbose=1, n_jobs=16)
rf.fit(X_train_plus_graph, ytrain_transformed_complique)
# Prediction for the training/validation set
y_train_pred_graph = rf.predict(X_train_plus_graph)
y_test_pred_graph= rf.predict(X_test_plus_graph)


print("---RandomForest avec graph = %s seconds ---" % (time.time() - start));start = time.time()


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:   49.6s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:  4.3min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    4.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    2.2s finished


---RandomForest sans ajout = 263.48823714256287 seconds ---


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:  1.1min
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:  4.9min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.6s
[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    3.5s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.3s


---RandomForest avec embedding = 297.68414855003357 seconds ---


[Parallel(n_jobs=16)]: Done 140 out of 140 | elapsed:    1.9s finished


In [17]:
print(classification_report(ytrain_transformed_complique,y_train_pred))
print(classification_report(ytest_transformed_complique,y_test_pred))
print("-------------------------------------------------------------------")
print(classification_report(ytrain_transformed_complique,y_train_pred_graph))
print(classification_report(ytest_transformed_complique,y_test_pred_graph))

cl=classification_report(ytest_transformed_complique,Y_test_pred,output_dict=True)
cl2=classification_report(ytest_transformed_complique,Y_test_pred_graph,output_dict=True)
recall_diff=(cl2["1"]["recall"]-cl["1"]["recall"])
f1_diff=(cl2["1"]["f1-score"]-cl["1"]["f1-score"])
print("rf recall diff = %.4f" % recall_diff)
print("rf f1-score diff = %.4f" % f1_diff)



              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       1.00      0.83      0.90      5968

    accuracy                           1.00   1037340
   macro avg       1.00      0.91      0.95   1037340
weighted avg       1.00      1.00      1.00   1037340

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.97      0.73      0.83      2145

    accuracy                           1.00    555719
   macro avg       0.99      0.86      0.92    555719
weighted avg       1.00      1.00      1.00    555719

-------------------------------------------------------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1031372
           1       1.00      0.83      0.91      5968

    accuracy                           1.00   1037340
   macro avg       1.00      0.92      0.95   1037340
weight

In [18]:
import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(Dense(32, input_shape=(len(xtrain_transformed_complique.columns),), activation='relu')),
model.add(BatchNormalization())
model.add(Dense(16, activation='relu')),
model.add(Dense(8, activation='relu')),
model.add(Dense(4, activation='relu')),
model.add(Dense(1, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.001) #optimizer
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy']) #metrics
earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=15, verbose=1,mode='auto', baseline=None, restore_best_weights=True)
history= model.fit(xtrain_transformed_complique.values, ytrain_transformed_complique.values, epochs = 100, batch_size=128, verbose = 1, validation_data=(xval_transformed_complique, yval_transformed_complique), callbacks = [earlystopper])
history_dict = history.history

Epoch 1/100
8105/8105 [==============================] - 17s 2ms/step - loss: 0.0382 - accuracy: 0.9938 - val_loss: 0.0370 - val_accuracy: 0.9941
Epoch 2/100
8105/8105 [==============================] - 16s 2ms/step - loss: 0.0355 - accuracy: 0.9942 - val_loss: 0.0364 - val_accuracy: 0.9941
Epoch 3/100
8105/8105 [==============================] - 13s 2ms/step - loss: 0.0355 - accuracy: 0.9942 - val_loss: 0.0365 - val_accuracy: 0.9941
Epoch 4/100
8105/8105 [==============================] - 13s 2ms/step - loss: 0.0355 - accuracy: 0.9942 - val_loss: 0.0365 - val_accuracy: 0.9941
Epoch 5/100
8105/8105 [==============================] - 13s 2ms/step - loss: 0.0355 - accuracy: 0.9942 - val_loss: 0.0366 - val_accuracy: 0.9941
Epoch 6/100
8105/8105 [==============================] - 13s 2ms/step - loss: 0.0355 - accuracy: 0.9942 - val_loss: 0.0367 - val_accuracy: 0.9941
Epoch 7/100
8105/8105 [==============================] - 13s 2ms/step - loss: 0.0355 - accuracy: 0.9942 - val_loss: 0.0363 -

In [21]:
print(len(X_train_plus_graph.columns))

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [19]:

############## a modifier
model2 = Sequential()
model2.add(Dense(32, input_shape=(len(X_train_plus_graph.columns),), activation='relu')),
model2.add(BatchNormalization())
model2.add(Dense(16, activation='relu')),
model2.add(Dense(8, activation='relu')),
model2.add(Dense(4, activation='relu')),
model2.add(Dense(1, activation='sigmoid'))
opt = tf.keras.optimizers.Adam(learning_rate=0.001) #optimizer
model2.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy']) #metrics
earlystopper = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0.001, patience=15, verbose=1,mode='auto', baseline=None, restore_best_weights=True)
history= model2.fit(X_train_plus_graph.values, hope_ytrain_concat.values, epochs = 100, batch_size=128, verbose = 1, validation_data=(hope_xval_concat, yval_transformed_complique), callbacks = [earlystopper])
history_dict = history.history

NameError: name 'hope_xtrain_concat' is not defined

In [ ]:

Y_train_pred =(model.predict(xtrain_transformed_complique) > 0.5).astype("int32")
Y_test_pred =(model.predict(xtest_transformed_complique) > 0.5).astype("int32")


y_train_pred_graph =(model2.predict(hope_xtrain_concat) > 0.5).astype("int32")
y_test_pred_graph =(model2.predict(hope_xtest_concat) > 0.5).astype("int32")


In [ ]:
print("NN sans modifications")
print(classification_report(ytrain_transformed_complique,y_train_pred))
print(classification_report(ytest_transformed_complique,y_test_pred))
print("-------------------------------------------------------------------")
print("NN avec des info de graph en plus")
print(classification_report(ytrain_transformed_complique,y_train_pred_graph))
print(classification_report(ytest_transformed_complique,y_test_pred_graph))

cl=classification_report(ytest_transformed_complique,Y_test_pred,output_dict=True)
cl2=classification_report(ytest_transformed_complique,Y_test_pred_graph,output_dict=True)
recall_diff=(cl2["1"]["recall"]-cl["1"]["recall"])
f1_diff=(cl2["1"]["f1-score"]-cl["1"]["f1-score"])
print("NN recall diff = %.4f" % recall_diff)
print("NN f1-score diff = %.4f" % f1_diff)

In [ ]:
not_too_long=False
if(not_too_long):#72h d'execution 
    from sklearn import svm
    start = time.time()
    svm_model = svm.SVC(kernel="rbf", gamma = 0.02, C=10)
    svm_model.fit(xtrain_transformed_complique,ytrain_transformed_complique)
    Y_train_pred=xgb.predict(xtrain_transformed_complique)
    Y_test_pred=xgb.predict(xtest_transformed_complique)
    print("---SVM sans modification = %s seconds ---" % (time.time() - start));start = time.time()
    print(classification_report(ytrain_transformed_complique,y_train_pred))
    print(classification_report(ytest_transformed_complique,y_test_pred))
    cl=classification_report(ytest_transformed_complique,Y_test_pred,output_dict=True))
    print("--------------------------------------------------------")
    ####
    svm_model.fit(X_train_plus_graph,ytrain_transformed_complique)
    y_train_pred_graph=xgb.predict(X_train_plus_graph)
    y_test_pred_graph=xgb.predict(X_test_plus_graph)
    print("---SVM avec plus d informations = %s seconds ---" % (time.time() - start));start = time.time()
    print(classification_report(ytrain_transformed_complique,y_train_pred_graph))
    print(classification_report(ytest_transformed_complique,y_test_pred_graph))
    cl2=classification_report(ytest_transformed_complique,y_test_pred_graph,output_dict=True)
    print("recall diff =",cl["1"]["recall"]-cl2["1"]["recall"])
    print("f1-score diff =",cl["1"]["f1-score"]-cl["1"]["f1-score"])
    print("--------------------------------------------------------")

In [ ]:
#print(dict_378)
print((M_le_maudit[0].shape),"x",len(M_le_maudit),"[",M_le_maudit[0][0],"]")
print(type(M_le_maudit[0]),"x",type(M_le_maudit),"[",type(M_le_maudit[0][0]),"]")

print((M_le_maudit_binary.shape),"[",M_le_maudit_binary[0][0:10],"]")
print(type(M_le_maudit_binary[0]),"x",type(M_le_maudit_binary),"[",type(M_le_maudit_binary[0][0]),"]")


print((M_le_binary.shape),"[",M_le_binary[0][0:10],"]")
print(type(M_le_binary[0]),"x",type(M_le_binary),"[",type(M_le_binary[0][0]),"]")
 

In [ ]:
print(cols[0])

In [ ]:
print(type(xtrain_transformed_complique))

In [ ]:
panda_test=xtrain_transformed_complique.copy()[["delta_amt"]]
print((panda_test))

In [ ]:
pair = dict_378["amt_hour"]
print((pair))
tmp_data_set=xtrain_transformed_complique.copy()[[pair[1],pair[2]]]
print((tmp_data_set).shape)
columns_list = tmp_data_set.columns.tolist()
print(columns_list)
x_train_not_normalized=(tmp_data_set[columns_list[0]].to_numpy())
y_train_not_normalized=(tmp_data_set[columns_list[1]].to_numpy())

x_train=NormalizeData(tmp_data_set[columns_list[0]].to_numpy())
y_train=NormalizeData(tmp_data_set[columns_list[1]].to_numpy())
print(len(tmp_data_set))

print(type(x_train))
print(x_train.shape)


In [ ]:
%matplotlib inline
plt.scatter(x_train, y_train)
plt.show()
plt.scatter(x_train_not_normalized, y_train_not_normalized)
plt.show()

In [ ]:
w,b =linear_regresion(tmp_data_set,boot_iter=10000)
boot_w,boot_b =lin_reg_bootstrap(tmp_data_set,nb_iter=10000)
np_dist=distances(tmp_data_set,w,b)

In [ ]:
print(np_dist.shape)
exp_np_dist=np.expand_dims(   ,axis=1)
print(exp_np_dist.shape)

In [ ]:
print("w= ",(w))
print("b= ",b)
print("boot_w= ",(boot_w))
print("boot_b= ",boot_b)
plt.ylim(-0.30, 1.3)  
plt.scatter(x_train,y_train)
plt.plot(x_train,x_train*w +b,"r")
plt.plot(x_train,x_train*boot_w +boot_b,"g")

plt.show()
#w = -406.7335090369642
#b = -0.44518610972447087

In [ ]:
np_test=xtest_transformed_complique.to_numpy()
normalized_np_test=NormalizeData(np_test)
print(np_test.shape)

In [ ]:
print(np_test[0])
print(normalized_np_test[0])
count=0
for obj in normalized_np_test:
    if count == 0:
        print (obj)
    count=count+1
print (count)

In [ ]:
#ajouter  une nouvelle colonne
'''
print(X_train_transformed.shape)
print(np.shape(moy))
moy=np.expand_dims(moy, axis=1)
print(moy.shape)
X_train_transformed =np.concatenate((X_train_transformed,moy),axis=1)
print(X_train_transformed.shape)
'''


In [ ]:
a=np.array([])
if(a.size ==0):
    a=np.array(list([0,0,0]))
b=np.array(list([1,1,1]))
print(np.shape(a))
a_=np.expand_dims(a,axis=1)
b_=np.expand_dims(b,axis=1)
print(np.shape(a_))
aray_test=np.concatenate((a,b),axis=0)

aray_test_=np.concatenate((a_,b_),axis=1)
print(aray_test_)
aray_test_=np.concatenate((aray_test_,b_*2),axis=1)
#aray_test=(a+b)

print(aray_test_)

In [ ]:
nd_aray_test_=aray_test_



In [ ]:
key_list=list(dict_378.keys())
for line in nd_aray_test_:
    #print(type(line),line.shape)
    print(line)
    for i in np.arange(len(line)):
        print (line[i])
        print (key_list[i])

In [ ]:
print(dict_378_binary.keys())

In [ ]:
print(dict_378_binary["category_day"] )

In [ ]:
dict_378_binary.items()

if(True):
    for pair in dict_378_binary.items():
        pair_dict[pair[0]][3]
        pair_dict[pair[0]][4]